In [1]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.logistic_regression

In [4]:
clients = utils.set_data()

In [5]:
clients = utils.set_data()
f1_before = []
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    #client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.F1)

In [6]:
f1_before

[0.7646612740141556,
 0.7757980697847068,
 0.7692307692307693,
 0.770508303975843,
 0.7710494033679044]

In [7]:
selected_model = Supported_modles.SGD_classifier

In [8]:
fedavg = Fedavg("global")
lr = 0.1
fedavg.init_global_model(lr, selected_model, clients[0].model,6)

In [9]:
model = clients[0].model
clients = utils.set_data()
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    client.init_empty_model(selected_model,model)

In [10]:
number_of_rounds = 20
batch_size = 0.1
epochs = 20
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        #X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!


ValueError: early_stopping should be False with partial_fit

In [40]:
f1_fedavg = []
for client in clients:
    #x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.8261363636363637, 0.8161116832589541, 0.8237442922374429, 0.8316605839416058, 0.81985670419652]


In [48]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.05260073373958474
Difference 0.0510898219222895
Difference 0.0668507936287912
Difference 0.049860684494647534
Difference 0.06269918166885946


In [236]:
selected_model = Supported_modles.gradient_boosting_classifier
client = utils.centralized_data()
client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
client.split_data()
client.train_model(selected_model)
print(client.F1)

0.8966526176460343
